# A partial pardot library of sorts
There are python libraries such as https://github.com/mneedham91/PyPardot4 which doesn't support the Salesforce Logins so you may want to get a fork of this such as https://github.com/jrkinley/PyPardot4 which does.

Here are some calls which are useful, not an exhaustive list.

Pardot API documentation is at https://developer.salesforce.com/docs/marketing/pardot/guide/overview.html 

See also https://www.sercante.com/labs/pardot-postman-collections/ for a Postman Collection which may prove useful

In [1]:
import requests
import pandas as pd
import numpy as np
import math
from io import StringIO
import os, os.path
import errno
import json
import functools
from datetime import datetime
import time
import urllib
import re
import csv 
from datetime import date



def PardotLogin():
    """Perform Pardot Auth
    
    Retain the PardotAuth for future use in later API calls
    
    Retuns: Access Token
    
    Raises: prints an error and the error message
    """
    url = "https://login.salesforce.com/services/oauth2/token"
    #url = "https://test.salesforce.com/services/oauth2/token" # Sandbox Login

    # If we are using the sercante JWT key, lets run it
    key = cfg['pardot_api_creds'].get('key')
    if key:
        claim = {
            'iss': cfg['pardot_api_creds']['PardotOath_Client_id'],
            'exp': int(time.time()) + 300,
            'aud': url,
            'sub': cfg['pardot_api_creds']['PardotOath_username'],
        }
        #print(claim)
        assertion = jwt.encode(claim, cfg['pardot_api_creds']['key'], 
                               algorithm='RS256', headers={'alg':'RS256'})
        #print(assertion)
        
        r = requests.post(url, data = {
            'grant_type': 'urn:ietf:params:oauth:grant-type:jwt-bearer',
            'assertion': assertion,
        })

        # print(r.json())
        if r.status_code == 200:
            accessToken = r.json().get('access_token')
            headers={
                'Authorization': 'Bearer '+accessToken,
                'Pardot-Business-Unit-Id': cfg['pardot_api_creds']['PardotBusinesUnitID']
            }

        payload= {
            'grant_type': 'urn:ietf:params:oauth:grant-type:jwt-bearer',
            'assertion': assertion
        }
        files = []
        headers = {
        }
        
        
        
    else: 
    
        payload= "grant_type=password&client_id={}&client_secret={}&username={}&password={}{}".format(
            urllib.parse.quote_plus(cfg['pardot_api_creds']['PardotOath_Client_id']),
            urllib.parse.quote_plus(cfg['pardot_api_creds']['PardotOath_client_secret']),
            urllib.parse.quote_plus(cfg['pardot_api_creds']['PardotOath_username']),
            urllib.parse.quote_plus(cfg['pardot_api_creds']['PardotOath_password']),
            urllib.parse.quote_plus(cfg['pardot_api_creds']['PardotOath_token'])
        )
        files = []
        headers = {
          'Content-Type': 'application/x-www-form-urlencoded',
        }
    #print(payload)
    response = requests.request("POST",
                                url,
                                headers=headers,
                                data=payload,
                                files=files)
    if response.status_code != 200:
        print(payload)
        print(response.text)
        raise Exception('POST /login/ {}'.format(response.status_code))

    return json.loads(response.text)['access_token']



def renew_pardot_access_token(func):
    """Renews Pardot Login
    
    Use as a decorator function - call @renew_pardot_access_token before function defintions to automatically attempt to renew the token
    https://medium.com/procurify-engineer/using-python-decorators-to-handle-expired-oauth-tokens-55e78316a188  
    
    Args: func: the function call to return to
    
    Returns: original results of function call
        
    
    """
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            # Invoke the code responsible for get a new token
            global access_token #we want to change this so lets pull in the global so we can effect that change
            access_token = PardotLogin()
            print("Renewing Access Token")
            # once the token is refreshed, we can retry the operation
            return func(*args, **kwargs)
    return wrapper


@renew_pardot_access_token
def PardotAccountRead():
    """Fetches Account information
    
    Returns: dict of account information
    """
    url = str(cfg['pardot_api_creds']['PardotURL']) + "/api/account/version/"+ str(cfg['pardot_api_creds']['apiversion']) +"/do/read?format=json"

    payload={}
    headers = {
        'Authorization': 'Bearer ' + access_token,
        'Pardot-Business-Unit-Id': cfg['pardot_api_creds']['PardotBusinesUnitID'],
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code != 200:
        raise Exception('GET /emailtempateread/ {}'.format(response.status_code) + format(url))
    response_dict = json.loads(response.text)
    if response_dict['@attributes'].get('err_code'):
        if response_dict['@attributes']['err_code'] == '1':
            print(response_dict)
            raise Exception('GET /emailread/ {}'.format(response.status_code) + format(url))

    return response_dict


@renew_pardot_access_token
def PardotReadExportStatus():
    """Fetch Export Status
    
    Returns: DICT of status results 
    """
    url = str(cfg['pardot_api_creds']['PardotURL']) + "/api/export/version/"+ str(cfg['pardot_api_creds']['apiversion']) +"/do/read/id/" + str(exportid) + "?format=json"

    payload = ""
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + access_token,
        'Pardot-Business-Unit-Id': cfg['pardot_api_creds']['PardotBusinesUnitID'],
    }
    #print(headers)
    response = requests.request("GET", url, headers=headers, data=payload)

    #print(response.text)
    return(json.loads(response.text))



@renew_pardot_access_token
def PardotGetExportFilesURLList():
    """Fetch Export files for download
    
    This will wait retrying every 5 minutes to get a file list
    
    Returns: DICT of status results 
    """
    status = PardotReadExportStatus()
    #print(status)
    while status['export']['state'] != "Complete":
        print(status['export']['state'])
        time.sleep(300)
        status = PardotReadExportStatus()
   
    #print(status['export']['isExpired'])
    if status['export']['isExpired'] == False:
        #print(status['export']['isExpired'])
        return status['export']['resultRefs']
            


        
@renew_pardot_access_token
def PardotDownloadFileURL(url):
    """Fetch .csv file 
    
    Args: url - a single URL string from PardotGetExportFilesURLList
    
    Returns: dataframe of data 
    """
    #url = str(cfg['pardot_api_creds']['PardotURL']) + "/api/export/version/"+ str(cfg['pardot_api_creds']['apiversion']) +"/do/downloadResults/id/" + str(exportid) + "/file/" + str(fileid)
    #print(url)
    payload={}
    headers = {
        'Authorization': 'Bearer ' + access_token,
        'Pardot-Business-Unit-Id': cfg['pardot_api_creds']['PardotBusinesUnitID'],
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    if response.status_code != 200:
        raise Exception('GET /downloadResults/ {}'.format(response.status_code) + format(url))

    csvdata = StringIO(response.text)

    #df = df.append(pd.read_csv(csvdata, sep=","))
    return pd.read_csv(csvdata, sep=",", low_memory=False)
    
        

@renew_pardot_access_token
def PardotDownloadFile():
    """Fetch .csv file 
    
    uses exportid global scoped variable

    Returns: dataframe of data 
    """    
    url = str(cfg['pardot_api_creds']['PardotURL']) + "/api/export/version/"+ str(cfg['pardot_api_creds']['apiversion']) +"/do/downloadResults/id/" + str(exportid) + "/file/" + str(fileid)
    print(url)
    payload={}
    headers = {
        'Authorization': 'Bearer ' + access_token,
        'Pardot-Business-Unit-Id': cfg['pardot_api_creds']['PardotBusinesUnitID'],
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    if response.status_code != 200:
        raise Exception('GET /downloadResults/ {}'.format(response.status_code) + format(url))

    csvdata = StringIO(response.text)

    #df = df.append(pd.read_csv(csvdata, sep=","))
    return pd.read_csv(csvdata, sep=",")
    
    
@renew_pardot_access_token
def PardotEmailRead():
    """Fetch specific email
    
    This uses email variable from global scope
    
    Returns: DICT of results 
    """
    url = str(cfg['pardot_api_creds']['PardotURL']) + "/api/email/version/"+ str(cfg['pardot_api_creds']['apiversion']) +"/do/read/id/"+str(int(email))+"?format=json"

    payload={}
    headers = {
        'Authorization': 'Bearer ' + access_token,
        'Pardot-Business-Unit-Id': cfg['pardot_api_creds']['PardotBusinesUnitID'],
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    #print(response)
    if response.status_code != 200:
        raise Exception('GET /emailread/ {}'.format(response.status_code) + format(url))         
    response_dict = json.loads(response.text)
    if response_dict['@attributes']['stat'] != 'ok':
        print(response_dict)
        raise Exception('GET /emailread/ {}'.format(response.status_code) + format(url))

    #print(response_dict)
    return response_dict
    
@renew_pardot_access_token
def PardotEmailtemplateRead():
    """Fetch specific email
    
    This reads the df datatable for values
    
    Returns: DICT of results 
    """    
    url = str(cfg['pardot_api_creds']['PardotURL']) + "/api/emailTemplate/version/"+ str(cfg['pardot_api_creds']['apiversion']) +"/do/read/id/"+str(int(df.loc[df.list_email_id==email].iloc[0]['email_template_id']))+"?format=json"

    payload={}
    headers = {
        'Authorization': 'Bearer ' + access_token,
        'Pardot-Business-Unit-Id': cfg['pardot_api_creds']['PardotBusinesUnitID'],
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code != 200:
        raise Exception('GET /emailtempateread/ {}'.format(response.status_code) + format(url))
    response_dict = json.loads(response.text)
    if response_dict['@attributes'].get('err_code'):
        if response_dict['@attributes']['err_code'] == '1':
            print(response_dict)
            raise Exception('GET /emailread/ {}'.format(response.status_code) + format(url))

    return response_dict

@renew_pardot_access_token
def PardotEmailtemplate2Read():
    """Fetch specific email
    
    This uses email variable from global scope
    
    Returns: DICT of results 
    """
    url = str(cfg['pardot_api_creds']['PardotURL']) + "/api/emailTemplate/version/"+ str(cfg['pardot_api_creds']['apiversion']) +"/do/read/id/"+str(int(emailtemplate))+"?format=json"

    payload={}
    headers = {
        'Authorization': 'Bearer ' + access_token,
        'Pardot-Business-Unit-Id': cfg['pardot_api_creds']['PardotBusinesUnitID'],
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code != 200:
        raise Exception('GET /emailtempateread/ {}'.format(response.status_code) + format(url))
    response_dict = json.loads(response.text)
    if response_dict['@attributes'].get('err_code'):
        if response_dict['@attributes']['err_code'] == '1':
            print(response_dict)
            raise Exception('GET /emailread/ {}'.format(response.status_code) + format(url))

    return response_dict


@renew_pardot_access_token
def PardotProspectbatchUpdate(body):
    """POSTs data
    
    Args: body - data to update
    
    Returns: DICT of response 
    """
    url = str(cfg['pardot_api_creds']['PardotURL']) + "/api/prospect/version/"+ str(cfg['pardot_api_creds']['apiversion']) +"/do/batchUpdate?format=json"

    payload = urllib.parse.urlencode({'prospects': str(body)})
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': 'Bearer ' + access_token,
        'Pardot-Business-Unit-Id': cfg['pardot_api_creds']['PardotBusinesUnitID'],
    }
    #print(url)
    #print(payload)
    #print(headers)
    response = requests.request("POST", url, headers=headers, data=payload)

    #print(response.text)
    return(json.loads(response.text))



    
PardotEmailtemplate2Readcache = dict() # lets use this as a crude cache
def PardotEmailtemplate2ReadCached(row):
    """Fetch specific emailtemplate
    
    When trying to get a lot of email templates repeatedly because we are guessing at what we need, we can use this so we don't have so many queries against the API server
    
    Returns: DICT of results 
    """     
    if row not in PardotEmailtemplate2Readcache:
            #print("Getting Record ", row)
            temp = PardotEmailtemplate2Read()
            if temp['@attributes']['stat'] == 'ok':
                PardotEmailtemplate2Readcache[row] = temp
            return temp
    return PardotEmailtemplate2Readcache[row]
